In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



In [6]:
data_aq = pd.DataFrame(pd.read_csv("../input/beijing-pm25-data-data-set/PRSA_data_2010.1.1-2014.12.31.csv"))
data_aq['datetime'] = pd.to_datetime(data_aq[['year', 'month','day', 'hour']])
data_aq = data_aq.drop(['year', 'month','day', 'hour'], axis = 1)

data_aq = data_aq[['datetime', 'DEWP', 'TEMP', 'PRES', 'cbwd', 'Iws', 'Is', 'Ir', 'pm2.5']].iloc[24:].reset_index().drop('index', axis = 1)
data_aq[['pm2.5']] = data_aq[['pm2.5']].interpolate(method='linear')

data_aq = data_aq.dropna().reset_index()

six_df = pd.DataFrame()
twelve_df = pd.DataFrame()
tf_df = pd.DataFrame()
for i in range(len(data_aq)):
    six_df = pd.concat([six_df,pd.DataFrame({'6hr_avg':[sum(data_aq['pm2.5'].iloc[i:i+6])/6]}, index = [i+6])])
    twelve_df = pd.concat([twelve_df,pd.DataFrame({'12hr_avg':[sum(data_aq['pm2.5'].iloc[i:i+12])/12]}, index = [i+12])])
    tf_df = pd.concat([tf_df,pd.DataFrame({'24hr_avg':[sum(data_aq['pm2.5'].iloc[i:i+24])/24]}, index = [i+24])])
    
data_aq = pd.merge(data_aq, six_df, left_index=True, right_index=True)
data_aq = pd.merge(data_aq, twelve_df, left_index=True, right_index=True)
data_aq = pd.merge(data_aq, tf_df, left_index=True, right_index=True)

data_aq = pd.merge(data_aq,pd.get_dummies(data_aq['cbwd']), left_index=True, right_index=True)

data_aq.head()

In [7]:
ax = data_aq['pm2.5'].plot.hist(bins = 200, figsize = (12,8))

In [8]:
ax = data_aq.plot(x = 'datetime', y = 'pm2.5', figsize = (12,8))

In [9]:
data_aq[['pm2.5']].mean()



In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_aq[['DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir','6hr_avg', '12hr_avg', '24hr_avg','NE','NW','SE']], data_aq[['pm2.5']], test_size=0.2)

In [11]:
import statsmodels.api as sm
import numpy as np

model = sm.OLS(np.asarray(y_train), np.asarray(X_train))
results = model.fit()
print(results.summary())


In [12]:
from sklearn.metrics import mean_squared_error

y_pred = results.predict(np.asarray(X_test))
mse = mean_squared_error(np.asarray(y_test), y_pred)
rmse = mse**0.5
print(rmse)

In [14]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
import pickle

dtest = xgb.DMatrix(X_test, y_test, feature_names=X_test.columns)
dtrain = xgb.DMatrix(X_train, y_train,feature_names=X_train.columns)

param = {'verbosity':1, 
         'objective':'reg:squarederror',
         'booster':'gblinear',
         'eval_metric' :'rmse',
         'learning_rate': 1}

evallist = [(dtrain, 'train')]

num_round = 20
bst = xgb.train(param, dtrain, num_round, evallist)

y_pred = bst.predict(dtest)

mse = mean_squared_error(np.asarray(y_test), y_pred)
rmse = mse**0.5
print(rmse)

dbfile = open('AirQuality.sav', 'wb')
# source, destination
pickle.dump(bst, dbfile)                     
dbfile.close()

